In [ ]:
import sys
sys.path.insert(0, './PythonFiles/')

import pandas as pd
import pickle
from Classifiers import Classifiers
from Iterative_Classif import Iterative_Classif
from Iterative_Classif2 import Iterative_Classif2
from Iterative_Classif_MIN import Iterative_Classif_MIN
import multiprocessing as mp

In [ ]:
def classify(Test_X, Test_Y, Train_X, Train_Y):

    classif = Classifiers(Train_X, Train_Y, Test_X, Test_Y)

    manager = mp.Manager()
    classifiers = manager.dict()

    p1 = mp.Process(target=classif.Train_NB, args=(classifiers,))
    p2 = mp.Process(target=classif.Train_SVM, args=(classifiers,))
    p3 = mp.Process(target=classif.Train_RandomForest, args=(classifiers,))
    p4 = mp.Process(target=classif.Train_LogReg, args=(classifiers,))

    p1.start()
    p2.start()
    p3.start()
    p4.start()

    p1.join()
    p2.join()
    p3.join()
    p4.join()

    classif.assign_classif(classifiers)
    return classif

In [ ]:
def openFile(filePath):
    try:
        with open(f'{filePath}.pkl', 'rb') as f:
            iterclass = pickle.load(f)
    except:
        raise Exception("Incorrect file path")

    try:
        with open(f'{filePath}Data.pkl', 'rb') as f:
            data = pickle.load(f)

        iterclass.score_log = data['score_log']
        iterclass.best_classifier_log = data['best_classifier_log']
        if 'best_classifier' in data.keys():
            iterclass.best_classif = data['best_classifier']
    except:
        print("Data File Not Present")

    return iterclass

def printPerformance(iterclass):
    Test_X, Test_Y, Train_X, Train_Y = iterclass.Test_X, iterclass.Test_Y, iterclass.Train_X, iterclass.Train_Y

    classif = classify(Test_X, Test_Y, Train_X, Train_Y)
    classif.print_reports()

    return classif

In [ ]:
filePath = "Models/oneStep500_YearDiff"
iterclass = openFile(filePath)

In [ ]:
printPerformance(iterclass)